# Real rag

llm setup

In [ ]:
from langchain_openai import ChatOpenAI


DEEPINFRA_API_TOKEN = "xxx"  # api key
DEEPINFRA_BASE_URL = "https://api.deepinfra.com/v1/openai"
def get_chat_model(model_name: str = "meta-llama/Llama-3.3-70B-Instruct-Turbo", _temp: float = 0) -> ChatOpenAI:
    return ChatOpenAI(
        api_key=DEEPINFRA_API_TOKEN,
        base_url=DEEPINFRA_BASE_URL,
        model=model_name,
        temperature= _temp
    )

llm = get_chat_model()

milvus lite vector store

In [26]:
from pymilvus import MilvusClient
client = MilvusClient("./rag_demo1.db")


In [33]:
# from langchain.embeddings import HuggingFaceEmbeddings
from langchain_community.embeddings import DeepInfraEmbeddings
from langchain_milvus import Milvus

# embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
embedding_model = DeepInfraEmbeddings(model_id="BAAI/bge-en-icl", deepinfra_api_token=DEEPINFRA_API_TOKEN)
URI = "./rag_demo2.db"
vector_store = Milvus(
    embedding_function=embedding_model,
    connection_args={"uri": URI, "token": "root:Milvus", "db_name": "milvus_demo"},
    index_params={"index_type": "FLAT", "metric_type": "L2"},
    collection_name="test_collection",
    collection_description="This is a testing collection for Rag demo",
)


Delete old docs

In [28]:
vector_store.delete(ids=["1", "2"])

True

Add documents

In [4]:
from langchain_core.documents import Document

document_1 = Document(
    page_content="Which campaign had the maximum impressions on 1 jan 2025?", 
    metadata={
        "ref_id": "query_000",
        "query_path" : "query_store/query_000.json"
        }
    )

document_2 = Document(
    page_content="What is the total number of campaigns with a conversion rate above 4/% in may 2025.",
    metadata={
        "ref_id": "query_001",
        "query_path" : "query_store/query_001.json"
        }
    )


Add documents from storage

In [28]:
from pathlib import Path
import json

def load_json_files(base_dir: str):
    json_paths = Path(base_dir).rglob("*.json")
    documents = []

    for path in json_paths:
        with open(path, "r") as f:
            data = json.load(f)
            question = data.get("query", "").strip()
            if not question:
                continue

            metadata = {
                "ref_id": data.get("ref_id"),
                "query_path": str(path),
            }

            doc = Document(page_content=question, metadata=metadata)
            documents.append(doc)
    
    return documents

In [29]:
base_dir = "/home/prakhar/luke-dev/txt2sql_methods/RaTsql/notebooks/metadata_query_store"
docs = load_json_files(base_dir)

In [30]:
vector_store.add_documents(docs, auto_id=True)

/home/prakhar/miniconda3/envs/txt2sql/lib/python3.12/site-packages/langchain_milvus/vectorstores/milvus.py:1215: UserWarning: No ids provided and auto_id is False. Setting auto_id to True automatically.
  warnings.warn(


[459744281146687488,
 459744281146687489,
 459744281146687490,
 459744281146687491,
 459744281146687492,
 459744281146687493,
 459744281146687494,
 459744281146687495,
 459744281146687496,
 459744281146687497,
 459744281146687498,
 459744281146687499,
 459744281146687500,
 459744281146687501,
 459744281146687502,
 459744281146687503,
 459744281146687504,
 459744281146687505,
 459744281146687506,
 459744281146687507,
 459744281146687508,
 459744281146687509,
 459744281146687510,
 459744281146687511,
 459744281146687512,
 459744281146687513,
 459744281146687514]

In [5]:
documents = [document_1, document_2]
ids = ["1", "2"]
vector_store.add_documents(documents=documents, ids=ids)


['1', '2']

Search store

In [25]:
results = vector_store.similarity_search_with_score(query="maximum impressions",k=1)
for doc, score in results:
    print(f"* [SIM={score}] {doc.page_content} [{doc.metadata}]")

* [SIM=27879.24609375] Which campaign had the maximum impressions on 1 jan 2025? [{'pk': 459744260041211904, 'query_path': '/home/prakhar/luke-dev/txt2sql_methods/RaTsql/notebooks/metadata_query_store/00/query_00000.json', 'ref_id': 'query_00000'}]


Load data from reff

In [16]:
import json

def get_shard_dir(ref_id):
    num = int(ref_id.split('_')[1])
    shard = f"{num // 1000:02}"
    return f"query_store/{shard}"

def load_query_info(path):
    with open(path, "r") as f:
        return json.load(f)
    
shard_dir = get_shard_dir(doc.metadata["ref_id"])
info = load_query_info(f"{shard_dir}/{doc.metadata["ref_id"]}.json")
sql = info["sql"]
instructions = info["instructions"]

In [17]:
print(f"User Query : {doc.page_content}\nSQL : {sql}\nINSTRUCTIONS : {instructions}")

User Query : Which campaign had the maximum impressions on 1 jan 2025?
SQL : SELECT gcd.campaign_id, gcd.campaign_name, SUM(gcp.metrics_impressions) AS total_impressions FROM advida-data-pipeline.manager_dataset_preprod_updated.google_ads_campaign_performance_2643649617 gcp JOIN advida-data-pipeline.manager_dataset_preprod_updated.google_ads_campaign_details_2643649617 gcd ON gcp.campaign_id = gcd.campaign_id WHERE gcp.customer_id = 8870377244 AND gcp.segments_date = '2025-01-01' GROUP BY gcd.campaign_id, gcd.campaign_name ORDER BY total_impressions DESC LIMIT 1
INSTRUCTIONS : nan
